In [ ]:
# 사용할 라이브러리

import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings(action='ignore')

import datetime 
import importlib
import json
from datetime import datetime

import webbrowser

!apt install chromium-chromedriver
!pip install selenium

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

from bs4 import BeautifulSoup
import numpy as np

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (85.0.4183.83-0ubuntu0.18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# 크롤링에 사용할 함수 구현

genre_dic = {}

def makeUrl(dateList,i, kinds):
  if kinds == 'playstore': 
    url = 'https://www.mobileindex.com/app/get_rank_all?rt=r&mk=2&c=kr&t=app&rs=100&d=' + dateList[i]
  else:
    url = 'https://www.mobileindex.com/app/get_rank_all?rt=r&mk=1&c=kr&t=app&rs=100&d=' + dateList[i]
  return url

def getGenreUrl(url):
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  genreURL = soup.select('div.item-info > a')
  return genreURL

def get_genre(url,appname):
  if appname in genre_dic:
    return genre_dic[appname]
  dr = webdriver.Chrome('chromedriver',options=options)
  dr.get(url)
  ht = dr.page_source
  so = BeautifulSoup(ht, 'html.parser')
  genre = so.select('tr.text-center')[0].text
  index1 = genre.find('\n',1)
  if genre[index1+1] == '\n':
    genre_dic[appname] = '기타'
    return '기타'
  index2 = genre.find('\n',14)
  genre_dic[appname] = genre[(index1+1):index2]
  return genre[(index1+1):index2]
  
def separateAppRank(rank):
  free_rank = []
  pay_rank = []
  sales_rank = [] 
  for j in range(len(rank)):
    if len(rank[j]) == 100:
      free_rank.append(rank[j][0::2])
    elif len(rank[j]) == 150 or len(rank[j]) == 300:
      free_rank.append(rank[j][0::3])
  return free_rank

def getAppRank(url, year):
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  notices = soup.select('span.appname')
  
  tmp = []
  for n in notices:
    tmp.append(n.text.strip())
  return tmp

In [ ]:
# 크롤링

from tqdm import tqdm
#2019
appRank_p = []
appRank_a = []
date_2019 = ['2019-01-31','2019-02-28','2019-03-31','2019-04-30','2019-05-31','2019-06-30','2019-07-31','2019-08-31','2019-09-30','2019-10-31','2019-11-30','2019-12-31']

date_size = len(date_2019)
kinds = ['playstore']


for kind in kinds:
  appRank = []
  appGenre = []
  for i in tqdm(range(date_size)):
    url = makeUrl(date_2019,i,kind)
    tmp = getAppRank(url,2019)
    appRank.append(tmp)
    genreURL = getGenreUrl(url)
    genreList = []
    for j in range(len(genreURL)):
      genreList.append(get_genre('https://www.mobileindex.com/'+genreURL[j]['href'],tmp[j]))
    appGenre.append(genreList)
    print(kind,' data of ',date_2019[i],' is done')
  if kind == 'playstore': 
    appRank_p = appRank[:]

  p_appRank_free_2019 = separateAppRank(appRank)
  print('p_appRank is done')
  p_appGenre_free_2019 = separateAppRank(appGenre)
  print('p_appGenre is done') 



#2020
appRank = []
date_2020 = ['2020-01-31','2020-02-29','2020-03-31','2020-04-30','2020-05-31','2020-06-30','2020-07-31','2020-08-31', '2020-09-23']
date_size = len(date_2020)
kinds = ['playstore']

for kind in kinds:
  appRank = []
  appGenre = []
  for i in tqdm(range(date_size)):
    url = makeUrl(date_2020,i,kind)
    tmp = getAppRank(url,2020)
    appRank.append(tmp)
    genreURL = getGenreUrl(url)
    genreList = []
    for j in range(len(genreURL)):
      genreList.append(get_genre('https://www.mobileindex.com/'+genreURL[j]['href'],tmp[j]))
    appGenre.append(genreList)
    print(kind,' date of ',date_2020[i],' is done')
  
  p_appRank_free_2020 = separateAppRank(appRank)
  print('p_appRank is done')
  p_appGenre_free_2020  = separateAppRank(appGenre)
  print('p_appGenre is done') 




  0%|          | 0/12 [00:00<?, ?it/s]

  8%|▊         | 1/12 [07:30<1:22:40, 450.96s/it]

playstore  data of  2019-01-31  is done




 17%|█▋        | 2/12 [09:47<59:26, 356.64s/it]  

playstore  data of  2019-02-28  is done




 25%|██▌       | 3/12 [11:53<43:06, 287.34s/it]

...

playstore  date of  2020-07-31  is done




 89%|████████▉ | 8/9 [12:24<01:26, 86.41s/it]

playstore  date of  2020-08-31  is done




100%|██████████| 9/9 [13:47<00:00, 91.91s/it]

playstore  date of  2020-09-23  is done
p_appRank is done
p_appGenre is done


In [ ]:
# 크롤링 -> csv로 변환

p_appRank_free = list()
for i in range(len(p_appRank_free_2019)):
  p_appRank_free.append(p_appRank_free_2019[i])
  p_appRank_free.append(p_appGenre_free_2019[i])

col = []
for i in range(12):
  col.append('19_'+str(i+1))
  col.append('19_'+str(i+1)+'_gen')

appRank_free_playstore_df = pd.DataFrame(p_appRank_free)
appRank_free_playstore_df = appRank_free_playstore_df.T
appRank_free_playstore_df.columns = col
appRank_free_playstore_df.to_csv('/content/gdrive/My Drive/빅콘/app/appRank_free_playstore_2019.csv', encoding='utf-8-sig')

p_appRank_free = list()
for i in tqdm(range(len(p_appRank_free_2020))):
  p_appRank_free.append(p_appRank_free_2020[i])
  p_appRank_free.append(p_appGenre_free_2020[i])

col = []
for i in tqdm(range(9)):
  col.append('20_'+str(i+1))
  col.append('20_'+str(i+1)+'_gen')

appRank_free_playstore_df = pd.DataFrame(p_appRank_free)
appRank_free_playstore_df = appRank_free_playstore_df.T
appRank_free_playstore_df.columns = col
appRank_free_playstore_df.to_csv('/content/gdrive/My Drive/빅콘/app/appRank_free_playstore_2020.csv', encoding='utf-8-sig')




100%|██████████| 9/9 [00:00<00:00, 57896.83it/s]


100%|██████████| 9/9 [00:00<00:00, 64527.75it/s]


In [ ]:
pd.read_csv('/content/gdrive/My Drive/빅콘/app/appRank_free_playstore_2019.csv')

,Unnamed: 0,19_1,19_1_gen,19_2,19_2_gen,19_3,19_3_gen,19_4,19_4_gen,19_5,19_5_gen,19_6,19_6_gen,19_7,19_7_gen,19_8,19_8_gen,19_9,19_9_gen,19_10,19_10_gen,19_11,19_11_gen,19_12,19_12_gen
0,0,국세청 홈택스,도구,Magic Booster - 제일 좋아하는 청리도구,도구,Samsung Notes,생산성,배달의민족,식음료,Snapchat,소셜,i-ONE Bank - IBK기업은행,금융,토스,금융,Google Play AR 서비스,도구,토스,금융,(New) 우리은행 우리WON뱅킹,금융,사용자 휴대폰 도우미 - Windows와 연결,생산성,트립스토어 - 해외 패키지 & 에어텔 특가 비교,여행 및 지역정보
1,1,TikTok 틱톡,동영상 플레이어/편집기,배달요기요,식음료,NH스마트뱅킹,금융,SRT - 수서고속철도(NEW),여행 및 지역정보,i-ONE Bank - IBK기업은행,금융,다이소 멤버십,라이프스타일,Samsung Notes,생산성,"T연락처 - 주소록 실시간 백업, 114 검색",도구,(New) 우리은행 우리WON뱅킹,금융,SNOW 스노우,사진,토스,금융,Netflix(넷플릭스),엔터테인먼트
2,2,Netflix(넷플릭스),엔터테인먼트,"커넥츠: 무료 공부질문 앱-500만 다운로드 (문제해설, 노하우, 과외, 인강)",교육,삼성 음성 녹음,도구,Super Magic Cleaner - 안드로이드 클리너 휴대폰 쿨러 안티바이러스,도구,국세청 홈택스,도구,토스,금융,카카오톡 KakaoTalk,커뮤니케이션,홈플러스 더 클럽,쇼핑,wavve(웨이브) - 재미의 파도를 타다!,엔터테인먼트,토스,금융,Samsung Notes,생산성,Instagram,소셜
3,3,NH스마트뱅킹,금융,카카오톡 KakaoTalk,커뮤니케이션,카카오톡 KakaoTalk,커뮤니케이션,네이버 - NAVER,도서/참고자료,배달의민족,식음료,호갱노노,부동산/홈 인테리어,삼성 음성 녹음,도구,V3 Mobile Plus 2.0,도구,스마트주택금융,금융,KB국민은행 스타뱅킹,금융,(New) 우리은행 우리WON뱅킹,금융,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑
4,4,카카오뱅크 - 같지만 다른 은행,금융,"Super Junk Cleaner - 휴대폰 쿨러, 안드로이드 클리너, 배터리 절약",도구,토스,금융,카카오톡 KakaoTalk,커뮤니케이션,네이버 - NAVER,도서/참고자료,카카오톡 KakaoTalk,커뮤니케이션,네이버 - NAVER,도서/참고자료,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑,VG 기업용 SDK 2.0,도구,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑,Wish - 쇼핑은 즐겁다,쇼핑,(New) 우리은행 우리WON뱅킹,금융
5,5,카카오톡 KakaoTalk,커뮤니케이션,"네이버 지도, 내비게이션",여행 및 지역정보,카카오뱅크 - 같지만 다른 은행,금융,NH스마트뱅킹,금융,Samsung Smart Switch Mobile,도구,네이버 - NAVER,도서/참고자료,호갱노노,부동산/홈 인테리어,배달요기요,식음료,카카오톡 KakaoTalk,커뮤니케이션,Wish - 쇼핑은 즐겁다,쇼핑,에이블리 - 셀럽마켓 모음앱,쇼핑,배달의민족,식음료
6,6,V3 Mobile Plus 2.0,도구,V3 Mobile Plus 2.0,도구,네이버 - NAVER,도서/참고자료,국세청 홈택스,도구,일기 예보 2020 - 실시간 날씨,날씨,Samsung Smart Switch Mobile,도구,Samsung Smart Switch Mobile,도구,SRT - 수서고속철도(NEW),여행 및 지역정보,Instagram,소셜,사용자 휴대폰 도우미 - Windows와 연결,생산성,Instagram,소셜,배달요기요,식음료
7,7,원더쇼핑:인기쇼핑몰/핫딜/여행쿠폰/웹툰/위메프투어/위메프페이,쇼핑,네이버 - NAVER,도서/참고자료,Samsung Smart Switch Mobile,도구,Samsung Smart Switch Mobile,도구,NH스마트뱅킹,금융,V3 Mobile Plus 2.0,도구,TikTok 틱톡,동영상 플레이어/편집기,쿠팡 (Coupang),쇼핑,Samsung Smart Switch Mobile,도구,어카운트인포-계좌정보통합관리,금융,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑,네이버 - NAVER,도서/참고자료
8,8,쿠팡 (Coupang),쇼핑,NH스마트뱅킹,금융,V3 Mobile Plus 2.0,도구,SODA 소다 - Natural Beauty Camera,사진,밴드,소셜,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑,Wish - 쇼핑은 즐겁다,쇼핑,"네이버 지도, 내비게이션",여행 및 지역정보,네이버 - NAVER,도서/참고자료,Samsung Smart Switch Mobile,도구,NH콕뱅크,금융,쿠팡 (Coupang),쇼핑
9,9,네이버 - NAVER,도서/참고자료,Samsung Smart Switch Mobile,도구,"미세미세 - 미세먼지, WHO기준, 알람, 위젯, 날씨, 지도",날씨,쿠팡 (Coupang),쇼핑,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑,Instagram,소셜,배달의민족,식음료,T map - 내비게이션 / 지도,지도/내비게이션,V3 Mobile Plus 2.0,도구,Instagram,소셜,트립스토어 - 해외 패키지 & 에어텔 특가 비교,여행 및 지역정보,"네이버 지도, 내비게이션",여행 및 지역정보


In [ ]:
pd.read_csv('/content/gdrive/My Drive/빅콘/app/appRank_free_playstore_2020.csv')

,Unnamed: 0,20_1,20_1_gen,20_2,20_2_gen,20_3,20_3_gen,20_4,20_4_gen,20_5,20_5_gen,20_6,20_6_gen,20_7,20_7_gen,20_8,20_8_gen,20_9,20_9_gen
0,0,국세청 홈택스,도구,우체국쇼핑,쇼핑,경기지역화폐,금융,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑,TikTok 틱톡,동영상 플레이어/편집기,TikTok 틱톡,동영상 플레이어/편집기,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑,ZOOM Cloud Meetings,비즈니스,ZOOM Cloud Meetings,비즈니스
1,1,Instagram,소셜,"코백-코로나 100m 근접시 푸시알림, 확진자 경로추적, 코로나19, 우한폐렴 정...",의료,ZOOM Cloud Meetings,비즈니스,국세청 홈택스,도구,국세청 홈택스,도구,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑,TikTok 틱톡,동영상 플레이어/편집기,BBQ치킨,식음료,건강상태 자가진단(교육부),건강/운동
2,2,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑,공영쇼핑 - 공영홈쇼핑,쇼핑,Netflix(넷플릭스),엔터테인먼트,Android Auto - 호환되는 차량 및 헤드유닛용,자동차,"하루날씨 - 한눈에 필수 정보 AI안내, 첫화면, 실시간, 예보, 미세먼지, WHO...",날씨,Clean Booster,도구,오늘의집 - 700만이 선택한 No.1 인테리어 필수앱,부동산/홈 인테리어,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑
3,3,NH스마트뱅킹,금융,Forest: 집중하기,생산성,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑,Google Duo - 고화질 화상 통화,커뮤니케이션,당근마켓 - 우리 동네 중고 직거래 벼룩장터,쇼핑,FaceApp,사진,Netflix(넷플릭스),엔터테인먼트,TikTok 틱톡,동영상 플레이어/편집기,SPOTV NOW(스포티비 나우) - 프리미엄 스포츠 생중계,스포츠
4,4,쿠팡 (Coupang),쇼핑,코로나맵 - 신종 코로나 바이러스 현황,건강/운동,Instagram,소셜,내 곁에 국민연금,금융,네이버 - NAVER,도서/참고자료,Netflix(넷플릭스),엔터테인먼트,ZOOM Cloud Meetings,비즈니스,Netflix(넷플릭스),엔터테인먼트,반올림피자샵,식음료
5,5,(New) 우리은행 우리WON뱅킹,금융,Amway,비즈니스,네이버 - NAVER,도서/참고자료,ZOOM Cloud Meetings,비즈니스,KB국민은행 스타뱅킹,금융,오늘의집 - 700만이 선택한 No.1 인테리어 필수앱,부동산/홈 인테리어,Samsung Smart Switch Mobile,도구,배달의민족,식음료,TikTok 틱톡,동영상 플레이어/편집기
6,6,네이버 - NAVER,도서/참고자료,Instagram,소셜,배달의민족,식음료,Instagram,소셜,카카오톡 KakaoTalk,커뮤니케이션,Action Cleaner,도구,카카오톡 KakaoTalk,커뮤니케이션,"Windy.com - 바람, 파도 및 태풍 예보",날씨,X Icon Changer - Customize App Icon & Shortcut,맞춤 설정
7,7,카카오톡 KakaoTalk,커뮤니케이션,대한민국 청와대,생산성,Samsung Smart Switch Mobile,도구,경기지역화폐,금융,Samsung Smart Switch Mobile,도구,Samsung Smart Switch Mobile,도구,"여기어때 - 호텔, 모텔, 펜션, 캠핑, 게하, 액티비티 최저가 예약!",여행 및 지역정보,하나알리미,금융,Netflix(넷플릭스),엔터테인먼트
8,8,Netflix(넷플릭스),엔터테인먼트,농협몰,쇼핑,오늘의집 - 700만이 선택한 No.1 인테리어 필수앱,부동산/홈 인테리어,Netflix(넷플릭스),엔터테인먼트,V3 Mobile Plus 2.0,도구,카카오톡 KakaoTalk,커뮤니케이션,토스,금융,전자출입명부(시설용) 보건복지부,건강/운동,쿠팡이츠 - 맛있는 음식을 빠르고 편하게,식음료
9,9,V3 Mobile Plus 2.0,도구,Wish - 쇼핑은 즐겁다,쇼핑,카카오톡 KakaoTalk,커뮤니케이션,카카오톡 KakaoTalk,커뮤니케이션,ZOOM Cloud Meetings,비즈니스,ZOOM Cloud Meetings,비즈니스,V3 Mobile Plus 2.0,도구,오늘의집 - 700만이 선택한 No.1 인테리어 필수앱,부동산/홈 인테리어,오늘의집 - 700만이 선택한 No.1 인테리어 필수앱,부동산/홈 인테리어
